# fasthtml

> High-level FastHTML integration for SSE with HTMX

In [ ]:
#| default_exp integrations.fasthtml

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from typing import Optional, List, Dict, Any, Callable, Union
from dataclasses import dataclass, field
import asyncio

# Import core components
from cjm_fasthtml_sse.core.broadcast import (
    SSEBroadcastManager,
    BroadcastMessage,
    BroadcastEventType
)
from cjm_fasthtml_sse.core.streaming import (
    sse_broadcast_stream,
    SSEStreamConfig,
    format_sse_message,
    OOBUpdate,
    format_oob_updates
)
from cjm_fasthtml_sse.core.decorators import (
    sse_endpoint,
    broadcast_action,
    sse_generator_endpoint
)
from cjm_fasthtml_sse.components.builders import (
    SSEConfig,
    create_sse_attrs,
    create_oob_attrs,
    MultiUpdateBuilder,
    generate_sse_cleanup_script,
    get_htmx_sse_extension_url
)
from cjm_fasthtml_sse.components.monitors import (
    MonitorConfig,
    generate_monitor_script,
    generate_simple_monitor,
    ConnectionState,
    create_default_status_indicators
)

# FastHTML imports
try:
    from fasthtml.common import (
        FastHTML, Script, Link, Div, Span, EventStream
    )
    FASTHTML_AVAILABLE = True
except ImportError:
    FASTHTML_AVAILABLE = False
    # Mock for testing
    class FastHTML: pass
    """Mock FastHTML class for testing when FastHTML is not installed"""
    class Script: pass
    """Mock Script element class for testing when FastHTML is not installed"""
    class Link: pass
    """Mock Link element class for testing when FastHTML is not installed"""
    class Div: pass
    """Mock Div element class for testing when FastHTML is not installed"""
    class Span: pass
    """Mock Span element class for testing when FastHTML is not installed"""
    def EventStream(
        s: str  # Stream content to return
    ) -> str:  # The stream content unchanged
        """Mock EventStream function for testing when FastHTML is not installed"""
        return s

## FastHTML SSE Integration

In [ ]:
#| export
@dataclass
class FastHTMLSSEConfig:
    """
    Configuration for FastHTML SSE integration.
    
    Attributes:
        app: FastHTML application instance
        prefix: URL prefix for SSE endpoints
        enable_monitor: Add connection monitoring scripts
        enable_cleanup: Add cleanup script for proper SSE closure
        enable_htmx_extension: Add HTMX SSE extension script
        monitor_debug: Enable debug logging in monitor
        broadcast_history: Number of messages to keep in history
        default_stream_config: Default configuration for SSE streams
    """
    app: Any  # FastHTML app
    prefix: str = "/sse"
    enable_monitor: bool = True
    enable_cleanup: bool = True
    enable_htmx_extension: bool = True
    monitor_debug: bool = False
    broadcast_history: int = 50
    default_stream_config: Optional[SSEStreamConfig] = None

In [ ]:
#| export
class FastHTMLSSE:
    """
    Main integration class for SSE with FastHTML.
    
    This class provides a high-level API for adding SSE capabilities
    to FastHTML applications with minimal configuration.
    """
    
    def __init__(
        self,
        config: FastHTMLSSEConfig  # Configuration for the SSE integration
    ):
        """
        Initialize FastHTML SSE integration.
        
        Args:
            config: Configuration for the integration
        """
        self.config = config
        self.app = config.app
        self.prefix = config.prefix
        
        # Initialize broadcast manager
        self.broadcast_manager = SSEBroadcastManager(
            history_size=config.broadcast_history,
            debug=config.monitor_debug
        )
        
        # Default stream configuration
        self.default_stream_config = config.default_stream_config or SSEStreamConfig(
            heartbeat_interval=30.0,
            reconnect_time=3000
        )
        
        # Setup scripts and routes
        self._setup_scripts()
        self._setup_default_routes()
    
    def _setup_scripts(self):
        """Add necessary scripts to the app headers"""
        if not FASTHTML_AVAILABLE:
            return
        
        # Add HTMX SSE extension
        if self.config.enable_htmx_extension:
            # Find HTMX script index
            htmx_idx = -1
            for i, hdr in enumerate(self.app.hdrs):
                if hasattr(hdr, 'attrs') and 'src' in hdr.attrs:
                    if 'htmx' in hdr.attrs['src']:
                        htmx_idx = i
                        break
            
            # Insert SSE extension after HTMX
            sse_script = Script(src=get_htmx_sse_extension_url())
            if htmx_idx >= 0:
                self.app.hdrs.insert(htmx_idx + 1, sse_script)
            else:
                self.app.hdrs.append(sse_script)
        
        # Add cleanup script
        if self.config.enable_cleanup:
            cleanup = Script(code=generate_sse_cleanup_script())
            self.app.hdrs.append(cleanup)
    
    def _setup_default_routes(self):
        """Setup default SSE routes"""
        if not FASTHTML_AVAILABLE:
            return
        
        # Global broadcast endpoint
        @self.app.get(f"{self.prefix}/global")
        @sse_endpoint(broadcast_manager=self.broadcast_manager)
        async def stream_global_updates():
            """Global SSE endpoint for cross-tab synchronization"""
            pass
    
    async def broadcast(
        self,
        event_type: str,  # Type of event to broadcast
        data: Dict[str, Any],  # Data payload to broadcast
        target_ids: Optional[List[str]] = None  # Optional list of element IDs to update
    ) -> int:  # Number of clients successfully notified
        """
        Broadcast a message to all connected clients.
        
        Args:
            event_type: Type of event
            data: Data to broadcast
            target_ids: Optional list of element IDs to update
            
        Returns:
            Number of clients notified
        """
        return await self.broadcast_manager.broadcast(
            event_type, data, target_ids
        )
    
    def create_sse_element(
        self,
        element_id: str,  # ID for the HTML element
        endpoint: Optional[str] = None,  # SSE endpoint URL
        content: Any = None,  # Initial content for the element
        hidden: bool = False,  # Whether to hide the element
        **kwargs
    ):
        """
        Create an SSE-enabled element.
        
        Args:
            element_id: ID for the element
            endpoint: SSE endpoint (defaults to global)
            content: Initial content
            hidden: Whether to hide the element
            **kwargs: Additional attributes
            
        Returns:
            Configured element
        """
        if not FASTHTML_AVAILABLE:
            return {"id": element_id, "endpoint": endpoint}
        
        endpoint = endpoint or f"{self.prefix}/global"
        attrs = create_sse_attrs(endpoint, **kwargs)
        attrs['id'] = element_id
        
        if hidden:
            attrs['style'] = 'display: none;'
        
        return Div(content, **attrs) if content else Div(**attrs)
    
    def create_monitor(
        self,
        sse_element_id: str = "global-sse",  # ID for the SSE connection element
        status_element_id: str = "connection-status"  # ID for the status display element
    ):
        """
        Create connection monitor elements.
        
        Args:
            sse_element_id: ID for SSE connection element
            status_element_id: ID for status display element
            
        Returns:
            Tuple of (sse_element, status_element, monitor_script)
        """
        if not FASTHTML_AVAILABLE:
            return None, None, ""
        
        # Create SSE connection element
        sse_element = self.create_sse_element(
            sse_element_id,
            hidden=True
        )
        
        # Create status display element
        status_element = Div(
            Span("Initializing...", cls="sse-status"),
            id=status_element_id
        )
        
        # Generate monitor script
        if self.config.enable_monitor:
            monitor_config = MonitorConfig(
                sse_element_id=sse_element_id,
                status_element_id=status_element_id,
                debug=self.config.monitor_debug
            )
            
            # Get status indicators
            indicators = create_default_status_indicators("sse-status")
            
            monitor_script = Script(
                code=generate_monitor_script(monitor_config, indicators)
            )
        else:
            monitor_script = None
        
        return sse_element, status_element, monitor_script
    
    def create_oob_update(
        self,
        updates: List[tuple]  # List of (element_id, content, swap_type) tuples
    ) -> str:  # Formatted SSE message with OOB updates
        """
        Create an OOB update message.
        
        Args:
            updates: List of (element_id, content, swap_type) tuples
            
        Returns:
            Formatted SSE message with OOB updates
        """
        oob_updates = []
        for update in updates:
            element_id = update[0]
            content = update[1]
            swap_type = update[2] if len(update) > 2 else "innerHTML"
            
            oob_updates.append(
                OOBUpdate(element_id, str(content), swap_type)
            )
        
        html = format_oob_updates(oob_updates)
        return format_sse_message(html)
    
    def sse_route(
        self,
        path: str,  # Route path for the SSE endpoint
        message_filter: Optional[Callable] = None  # Optional filter function for messages
    ):
        """
        Decorator for creating SSE routes.
        
        Args:
            path: Route path
            message_filter: Optional filter function
            
        Returns:
            Route decorator
        """
        def decorator(
            func: Callable  # Route handler function
        ):
            """Apply SSE endpoint decorator to a route handler"""
            # Register route with app
            route_path = f"{self.prefix}{path}" if not path.startswith('/') else path
            
            @self.app.get(route_path)
            @sse_endpoint(
                broadcast_manager=self.broadcast_manager,
                config=self.default_stream_config,
                message_filter=message_filter
            )
            async def wrapper(*args, **kwargs):
                """Wrapper function for SSE route handler"""
                return await func(*args, **kwargs) if asyncio.iscoroutinefunction(func) else func(*args, **kwargs)
            
            return wrapper
        return decorator
    
    def action_route(
        self,
        path: str,  # Route path for the action endpoint
        event_type: Optional[str] = None,  # Event type to broadcast after action
        method: str = "post"  # HTTP method for the route
    ):
        """
        Decorator for creating action routes that broadcast.
        
        Args:
            path: Route path
            event_type: Event type to broadcast
            method: HTTP method
            
        Returns:
            Route decorator
        """
        def decorator(
            func: Callable  # Route handler function
        ):
            """Apply broadcast action decorator to a route handler"""
            # Get the route decorator based on method
            route_decorator = getattr(self.app, method.lower())
            
            @route_decorator(path)
            @broadcast_action(
                self.broadcast_manager,
                event_type=event_type
            )
            async def wrapper(*args, **kwargs):
                """Wrapper function for action route handler"""
                return await func(*args, **kwargs) if asyncio.iscoroutinefunction(func) else func(*args, **kwargs)
            
            return wrapper
        return decorator

## Helper Functions

In [ ]:
#| export
def setup_sse(
    app: Any,  # FastHTML application
    prefix: str = "/sse",  # URL prefix for SSE routes
    **kwargs  # Additional configuration options
) -> FastHTMLSSE:  # Configured FastHTMLSSE instance
    """Quick setup function for SSE in FastHTML apps."""
    config = FastHTMLSSEConfig(
        app=app,
        prefix=prefix,
        **kwargs
    )
    return FastHTMLSSE(config)

In [ ]:
#| export
def create_sse_page_template(
    sse: FastHTMLSSE,  # FastHTMLSSE instance
    title: str = "SSE Demo",  # Page title
    content: Any = None  # Page content
):
    "Create a basic page template with SSE setup."
    if not FASTHTML_AVAILABLE:
        return {"title": title, "content": content}
    
    # Create monitor elements
    sse_elem, status_elem, monitor_script = sse.create_monitor()
    
    return Div(
        # Hidden SSE connection
        sse_elem,
        
        # Status display
        Div(
            status_elem,
            cls="connection-status"
        ),
        
        # Main content
        Div(
            content or "Add your content here",
            id="main-content"
        ),
        
        # Monitor script
        monitor_script
    )

## Example Usage

In [ ]:
# Example: Basic setup
def example_basic_setup():
    if not FASTHTML_AVAILABLE:
        print("FastHTML not available, using mock")
        app = FastHTML()
    else:
        print("FastHTML available")
        app = FastHTML()
    
    # Setup SSE
    config = FastHTMLSSEConfig(
        app=app,
        prefix="/sse",
        enable_monitor=True,
        monitor_debug=True
    )
    
    sse = FastHTMLSSE(config)
    
    print(f"SSE Setup Complete:")
    print(f"  Prefix: {sse.prefix}")
    print(f"  Broadcast manager: {sse.broadcast_manager}")
    print(f"  Monitor enabled: {config.enable_monitor}")
    print()
    
    # Quick setup alternative
    sse2 = setup_sse(app, prefix="/events")
    print(f"Quick setup prefix: {sse2.prefix}")
    
    return sse

sse = example_basic_setup()

FastHTML available
SSE Setup Complete:
  Prefix: /sse
  Broadcast manager: <cjm_fasthtml_sse.core.broadcast.SSEBroadcastManager object>
  Monitor enabled: True

Quick setup prefix: /events


In [ ]:
# Example: Creating SSE elements
def example_sse_elements():
    if not FASTHTML_AVAILABLE:
        print("Using mock elements")
    
    # Create an SSE element
    elem = sse.create_sse_element(
        element_id="notifications",
        endpoint="/sse/notifications",
        content="Loading notifications..."
    )
    
    print("SSE Element created:")
    if FASTHTML_AVAILABLE:
        print(f"  Type: {type(elem)}")
        print(f"  ID: {elem.attrs.get('id')}")
        print(f"  Endpoint: {elem.attrs.get('sse-connect')}")
    else:
        print(f"  Mock element: {elem}")
    print()
    
    # Create monitor elements
    sse_elem, status_elem, script = sse.create_monitor(
        "global-sse",
        "status-display"
    )
    
    print("Monitor elements created:")
    print(f"  SSE element: {sse_elem is not None}")
    print(f"  Status element: {status_elem is not None}")
    print(f"  Monitor script: {script is not None or not sse.config.enable_monitor}")

example_sse_elements()

SSE Element created:
  Type: <class 'fastcore.xml.FT'>
  ID: notifications
  Endpoint: /sse/notifications

Monitor elements created:
  SSE element: True
  Status element: True
  Monitor script: True


In [ ]:
# Example: Broadcasting
async def example_broadcasting():
    # Send a broadcast
    count = await sse.broadcast(
        "notification",
        {"message": "Hello, world!", "timestamp": "2024-01-01T12:00:00"},
        target_ids=["notification-area"]
    )
    
    print(f"Broadcast sent to {count} clients")
    print()
    
    # Create OOB update
    oob_message = sse.create_oob_update([
        ("status", "<span>Active</span>", "innerHTML"),
        ("counter", "<strong>42</strong>", "innerHTML"),
        ("alert", "<div class='alert'>New message!</div>", "beforeend")
    ])
    
    print("OOB Update Message:")
    print(oob_message[:200] + "..." if len(oob_message) > 200 else oob_message)

await example_broadcasting()

[SSEBroadcastManager] No active connections to broadcast to
Broadcast sent to 0 clients

OOB Update Message:
data: <div id="status" hx-swap-oob="innerHTML"><span>Active</span></div><div id="counter" hx-swap-oob="innerHTML"><strong>42</strong></div><div id="alert" hx-swap-oob="beforeend"><div class='alert'>Ne...


In [ ]:
# Example: Route decorators
def example_route_decorators():
    # Mock route registration
    registered_routes = []
    
    # SSE route
    @sse.sse_route("/updates")
    async def stream_updates():
        """Stream updates to clients"""
        return lambda msg: msg.type == "update"
    
    registered_routes.append("/sse/updates")
    
    # Action route
    @sse.action_route("/api/create", event_type="item_created")
    async def create_item(data):
        """Create item and broadcast"""
        return {"id": "123", "name": data.get("name")}
    
    registered_routes.append("/api/create")
    
    print("Routes registered:")
    for route in registered_routes:
        print(f"  - {route}")
    
    print("\nNote: In a real FastHTML app, these would be actual routes")

example_route_decorators()

Routes registered:
  - /sse/updates
  - /api/create

Note: In a real FastHTML app, these would be actual routes


In [ ]:
# Example: Page template
def example_page_template():
    page = create_sse_page_template(
        sse,
        title="My SSE App",
        content="Welcome to the SSE demo!"
    )
    
    print("Page template created:")
    if FASTHTML_AVAILABLE:
        print(f"  Type: {type(page)}")
        print(f"  Has SSE element: {'global-sse' in str(page)}" if hasattr(page, '__str__') else "N/A")
    else:
        print(f"  Mock page: {page}")
    
    print("\nPage structure includes:")
    print("  - Hidden SSE connection element")
    print("  - Connection status display")
    print("  - Main content area")
    print("  - Monitor script (if enabled)")

example_page_template()

Page template created:
  Type: <class 'fastcore.xml.FT'>
  Has SSE element: True

Page structure includes:
  - Hidden SSE connection element
  - Connection status display
  - Main content area
  - Monitor script (if enabled)


## Testing

In [ ]:
# Test: Configuration
def test_configuration():
    app = FastHTML() if FASTHTML_AVAILABLE else type('MockApp', (), {})()
    
    config = FastHTMLSSEConfig(
        app=app,
        prefix="/test",
        enable_monitor=False,
        broadcast_history=100
    )
    
    assert config.prefix == "/test"
    assert config.enable_monitor == False
    assert config.broadcast_history == 100
    assert config.enable_cleanup == True  # Default
    
    print("✓ Configuration tests passed")

test_configuration()

✓ Configuration tests passed


In [ ]:
# Test: SSE integration
async def test_sse_integration():
    app = FastHTML() if FASTHTML_AVAILABLE else type('MockApp', (), {'hdrs': []})()
    
    sse = setup_sse(app, prefix="/events")
    
    assert sse.prefix == "/events"
    assert sse.broadcast_manager is not None
    
    # Test broadcasting
    count = await sse.broadcast("test", {"value": 42})
    assert count >= 0
    
    # Test OOB update creation
    oob = sse.create_oob_update([
        ("test-id", "content", "innerHTML")
    ])
    assert "test-id" in oob
    assert "innerHTML" in oob
    
    print("✓ SSE integration tests passed")

await test_sse_integration()

✓ SSE integration tests passed


In [ ]:
# Test: Helper functions
def test_helpers():
    app = FastHTML() if FASTHTML_AVAILABLE else type('MockApp', (), {'hdrs': []})()
    
    # Test setup_sse
    sse = setup_sse(app, enable_monitor=False)
    assert isinstance(sse, FastHTMLSSE)
    assert sse.config.enable_monitor == False
    
    # Test page template
    page = create_sse_page_template(sse, "Test Page")
    assert page is not None
    
    print("✓ Helper function tests passed")

test_helpers()

✓ Helper function tests passed


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()